In [10]:
# use python2

# use only one GPU device
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
os.system('echo $CUDA_VISIBLE_DEVICES')

0

In [11]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

In [12]:
config = tf.ConfigProto()

In [13]:
config.gpu_options.allow_growth = True

In [14]:
set_session(tf.Session(config=config))

In [21]:
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Dropout, UpSampling2D, Input, merge

In [16]:
from keras.regularizers import l2

In [17]:
weight_decay = 0.01

In [18]:
input_shape = (512, 512, 3)
img_input = Input(shape=input_shape)

In [22]:
x

<tf.Tensor 'dropout_8/cond/Merge:0' shape=(?, 32, 32, 4096) dtype=float32>

In [31]:
# block1 
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block1_conv1',
          kernel_regularizer=l2(weight_decay))(img_input)
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block1_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block1_pool')(x)

# block2
x = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block2_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block2_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block2_pool')(x)

# block3
x = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block3_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block3_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block3_conv3',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block3_pool')(x)

# block4
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block4_conv1',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block4_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block4_conv3',
          kernel_regularizer=l2(weight_decay))(x)
x4 = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block4_pool')(x)

# block5
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block5_conv1',
          kernel_regularizer=l2(weight_decay))(x4)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block5_conv2',
          kernel_regularizer=l2(weight_decay))(x)
x = Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', name='block5_conv3',
          kernel_regularizer=l2(weight_decay))(x)
x = MaxPool2D(pool_size=(2,2), strides=(2,2), name='block5_pool')(x)

# fully convolutional
x = Conv2D(filters=4096, kernel_size=(7,7), activation='relu', padding='same', name='fc1', 
           kernel_regularizer=l2(weight_decay))(x)
x = Dropout(0.5)(x)
x = Conv2D(filters=4096,kernel_size=(1,1), activation='relu', padding='same', name='fc2',
           kernel_regularizer=l2(weight_decay))(x)
x = Dropout(0.5)(x)

# segment
x = Conv2D(filters=21, kernel_size=(1,1), activation='softmax', padding='same',
           kernel_regularizer=l2(weight_decay))(x)

# upsample x 32
x = UpSampling2D(size=(32,32))(x)

# Add
# fully convolutional
y = Conv2D(filters=4096, kernel_size=(7,7), activation='relu', padding='same', name='fc1_', 
           kernel_regularizer=l2(weight_decay))(x4)
y = Dropout(0.5)(y)
y = Conv2D(filters=4096,kernel_size=(1,1), activation='relu', padding='same', name='fc2_',
           kernel_regularizer=l2(weight_decay))(y)
y = Dropout(0.5)(y)

# segment
y = Conv2D(filters=21, kernel_size=(1,1), activation='softmax', padding='same',
           kernel_regularizer=l2(weight_decay))(y)

# upsample x 16
y = UpSampling2D(size=(16,16))(y)

# merge
x = merge([x, y] , mode='sum')

/root/anaconda2/envs/py2.7_tf1.0/lib/python2.7/site-packages/ipykernel_launcher.py:74: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [32]:
# construct final model
model = Model(inputs=img_input, outputs=x)

In [33]:
# print model details (e.g. feature map's size)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 512, 512, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 512, 512, 64)  1792                                         
____________________________________________________________________________________________________
block1_conv2 (Conv2D)            (None, 512, 512, 64)  36928                                        
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 256, 256, 64)  0                                            
___________________________________________________________________________________________

In [28]:
??plot_model

In [34]:
# visualize
from keras.utils import plot_model
plot_model(model, to_file='./fcn16s.png', show_shapes=True, show_layer_names=True)

### fcn.png
https://github.com/amiltonwong/xl_dl/blob/master/keras_seg/fcn.png